# 부동산 분석
- 데이터: https://data.seoul.go.kr/dataList/OA-21275/S/1/datasetView.do
- 

In [1]:
import plotly

In [131]:
import pandas as pd
import numpy as np
import datetime

In [87]:
pd.set_option('display.max_columns', 500)

In [116]:
df_2022 = pd.read_csv('data/서울시 부동산 실거래가 정보_2022.csv', encoding='euc-kr')

In [117]:
df_2023 = pd.read_csv('data/서울시 부동산 실거래가 정보_2023.csv', encoding='euc-kr')

In [118]:
df = pd.concat([df_2022, df_2023]).reset_index(drop=True)

In [108]:
def make_building_code(row):
    use_columns = ['자치구명', '법정동명', '본번', '부번']
    is_nan = False
    for col in use_columns:
        is_nan = is_nan | pd.isna(row[col])
    if is_nan:
        return np.nan
    else:
        building_code = f"{row['자치구명']} {row['법정동명']} {str(int(row['본번']))}-{str(int(row['부번']))}"
        if building_code.endswith('-0'):
            building_code = building_code[:-2]
        building_code += ' ' + str(row[BUILDING_AREA]) + '(㎡)'
        return building_code
        
    building_code = []
    for col in use_columns:
        col_code = row[col]
        if not isinstance(col_code, str):
            col_code = str(int(col_code))
        building_code.append(col_code)
    building_code = '-'.join(building_code)  # ['a', 'b', 'c'] --> 'a-b-c'
    return building_code

In [124]:
BUILDING_AREA = '건물면적(㎡)'
BUILDING_CODE = '건물코드'

In [121]:
df['건물코드'] = df.apply(lambda x:make_building_code(x), axis=1)

In [122]:
df['건물코드']

0         중랑구 면목동 173-164 50.35(㎡)
1             광진구 자양동 860 75.03(㎡)
2         강남구 개포동 1165-16 50.28(㎡)
3          성동구 용답동 238-12 20.21(㎡)
4         강동구 천호동 167-153 29.01(㎡)
                    ...           
273798        금천구 시흥동 865 51.02(㎡)
273799     성북구 성북동 145-36 44.42(㎡)
273800      도봉구 창동 608-56 43.77(㎡)
273801      서초구 잠원동 55-7 242.16(㎡)
273802        은평구 신사동 370 84.96(㎡)
Name: 건물코드, Length: 273803, dtype: object

## Analysis

### 1. 거래건수

### 2. 집값 등락
- 하루전 대비, 일주일전 대비, 한달전 대비 집값은 얼마 올랐나
- 계산법: 특정 시점(or 구간)에 거래된 건물코드 리스트를 대상으로 가장 최근 거래된 건물(최대 n개월 or m년 이내) 대비 등락률을 계산
    - 건수가 많으면?
- 일단 효율성은 고려안하고 구현해보자

In [125]:
sum(df.groupby('건물코드').size() == 1)

14752

In [126]:
sum(df.groupby('건물코드').size() != 1)

35858

In [127]:
len(df)

273803

In [128]:
len(df['건물코드'].unique())

50611

In [153]:
compare_to_date = 20230601
target_from_date = 20230901
target_to_date = 20230914

In [162]:
compare_gap = 365

In [163]:
compare_from_date = datetime.datetime.strptime(str(compare_to_date), '%Y%m%d')
compare_from_date = int(datetime.datetime.strftime(compare_from_date - datetime.timedelta(days=compare_gap), format='%Y%m%d'))

In [158]:
df_prior = df[(df['계약일'] >= compare_from_date) & (df['계약일'] <= compare_to_date)]

In [159]:
df_target = df[(df['계약일'] >= target_from_date) & (df['계약일'] < target_to_date)]

In [166]:
target_building_data = df_target.groupby('건물코드').mean()['물건금액(만원)'].to_dict()

0         중랑구 면목동 173-164 50.35(㎡)
1             광진구 자양동 860 75.03(㎡)
2         강남구 개포동 1165-16 50.28(㎡)
3          성동구 용답동 238-12 20.21(㎡)
4         강동구 천호동 167-153 29.01(㎡)
                    ...           
273794        종로구 명륜2가 236 84.9(㎡)
273795        종로구 명륜2가 236 84.9(㎡)
273796                         NaN
273797    영등포구 영등포동7가 208 19.31(㎡)
273798        금천구 시흥동 865 51.02(㎡)
Name: 건물코드, Length: 112140, dtype: object

In [172]:
df_prior[df_prior['건물코드'] == building_code]

,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분,지번구분명,본번,부번,건물명,계약일,물건금액(만원),건물면적(㎡),토지면적(㎡),층,권리구분,취소일,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명,건물코드


In [174]:
for building_code, price in target_building_data.items():
    l = len(df_prior[df_prior['건물코드'] == building_code])
    if l:
        break

In [176]:
price

27500.0

In [175]:
df_prior[df_prior['건물코드'] == building_code]

,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분,지번구분명,본번,부번,건물명,계약일,물건금액(만원),건물면적(㎡),토지면적(㎡),층,권리구분,취소일,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명,건물코드
25971,2022,11680,강남구,10300,개포동,1.0,대지,1231.0,21.0,드림하우스,20220901,24500,17.34,11.9,2.0,NaN,NaN,2014.0,연립다세대,중개거래,서울 서초구,강남구 개포동 1231-21 17.34(㎡)
25976,2022,11680,강남구,10300,개포동,1.0,대지,1231.0,21.0,드림하우스,20220901,24500,17.34,11.9,2.0,NaN,NaN,2014.0,연립다세대,중개거래,서울 서초구,강남구 개포동 1231-21 17.34(㎡)
26201,2022,11680,강남구,10300,개포동,1.0,대지,1231.0,21.0,드림하우스,20220901,24500,17.34,11.9,2.0,NaN,NaN,2014.0,연립다세대,중개거래,서울 서초구,강남구 개포동 1231-21 17.34(㎡)


In [57]:
df_prior_price = df_prior.groupby([BUILDING_CODE, BUILDING_AREA]).mean()['물건금액(만원)']


In [58]:
df_posterior_price = df_posterior.groupby([BUILDING_CODE, BUILDING_AREA]).mean()['물건금액(만원)']

In [59]:
df_prior_price = pd.DataFrame(df_prior_price).reset_index()
df_prior_price = df_prior_price.rename(columns={'물건금액(만원)': '물건금액(만원)_prior'})

In [60]:
df_posterior_price = pd.DataFrame(df_posterior_price).reset_index()
df_posterior_price = df_posterior_price.rename(columns={'물건금액(만원)': '물건금액(만원)_posterior'})

In [63]:
df_merge = df_prior_price.merge(df_posterior_price, on=[BUILDING_CODE, BUILDING_AREA])

In [64]:
df_merge

,건물코드,건물면적(㎡),물건금액(만원)_prior,물건금액(만원)_posterior
0,강남구 개포동 1231-21,17.34,24500.0,27500.0
1,강남구 개포동 13-3,31.91,21325.0,19800.0
2,강남구 논현동 22-0,35.73,90000.0,87000.0
3,강남구 대치동 316-0,76.79,223300.0,231375.0
4,강남구 도곡동 467-6,138.66,260000.0,271000.0
...,...,...,...,...
343,중랑구 신내동 650-0,39.84,40250.0,39000.0
344,중랑구 신내동 661-0,59.76,52000.0,49500.0
345,중랑구 신내동 785-0,57.30,52500.0,55000.0
346,중랑구 신내동 816-0,84.65,80000.0,80000.0


In [79]:
df_merge['ratio'] = (df_merge['물건금액(만원)_posterior']/df_merge['물건금액(만원)_prior'] - 1)*100

In [96]:
df_merge['주소_구'] = df_merge.apply(lambda x:x['건물코드'].split('-')[0], axis=1)
df_merge['주소_동'] = df_merge.apply(lambda x:' '.join(x['건물코드'].split('-')[:2]), axis=1)

In [95]:
df_merge.groupby('주소').mean()['ratio'].sort_values()

주소
영등포구 대림동      -32.275132
영등포구 당산동6가    -27.272727
강동구 상일동       -17.961903
노원구 중계동       -16.908930
은평구 수색동       -15.447805
                 ...    
용산구 이태원동       43.020073
금천구 독산동       110.617267
용산구 문배동       125.423729
종로구 사직동       275.641026
종로구 내수동       285.520536
Name: ratio, Length: 132, dtype: float64

In [97]:
df_merge.groupby('주소_구').mean()['ratio'].sort_values()

주소_구
중랑구      -7.860449
구로구      -5.227198
동대문구     -5.017144
은평구      -4.451871
관악구      -3.096512
노원구      -2.205352
양천구      -1.691259
성북구      -0.539158
영등포구      0.635759
송파구       0.652155
강서구       0.885938
광진구       1.449003
동작구       1.755353
중구        2.578430
서초구       3.402614
강동구       3.963062
도봉구       4.926320
마포구       6.288729
강남구       7.597298
서대문구     14.999968
성동구      17.907724
강북구      22.207027
용산구      42.396614
금천구      44.444521
종로구     171.398770
Name: ratio, dtype: float64

In [100]:
df_merge[df_merge['주소_구'] == '종로구']

,건물코드,물건금액(만원)_prior,물건금액(만원)_posterior,ratio,주소,주소_구,주소_동
298,종로구-내수동-72-0,31400.0,208000.0,562.420382,종로구 내수동,종로구,종로구 내수동
299,종로구-내수동-75-0,29000.0,31500.0,8.620690,종로구 내수동,종로구,종로구 내수동
300,종로구-무악동-82-0,96726.0,117900.0,21.890702,종로구 무악동,종로구,종로구 무악동
301,종로구-사직동-9-0,39000.0,146500.0,275.641026,종로구 사직동,종로구,종로구 사직동
302,종로구-숭인동-2-1,95000.0,84000.0,-11.578947,종로구 숭인동,종로구,종로구 숭인동


In [106]:
df[df['건물코드'] == '종로구-내수동-72-0'].iloc[1]

접수연도                           2023
자치구코드                         11110
자치구명                            종로구
법정동코드                         11800
법정동명                            내수동
지번구분                            1.0
지번구분명                            대지
본번                             72.0
부번                              0.0
건물명                     경희궁의 아침 3단지
계약일                        20230807
물건금액(만원)                      31400
건물면적(㎡)                        37.5
토지면적(㎡)                       48.95
층                              16.0
권리구분                            NaN
취소일                             NaN
건축년도                         2004.0
건물용도                           오피스텔
신고구분                           중개거래
신고한 개업공인중개사 시군구명    서울 영등포구, 서울 종로구
건물코드                   종로구-내수동-72-0
Name: 5669, dtype: object

In [105]:
df[df['건물코드'] == '종로구-내수동-72-0'].iloc[0]

접수연도                          2023
자치구코드                        11110
자치구명                           종로구
법정동코드                        11800
법정동명                           내수동
지번구분                           1.0
지번구분명                           대지
본번                            72.0
부번                             0.0
건물명                      경희궁의아침3단지
계약일                       20230909
물건금액(만원)                    208000
건물면적(㎡)                     150.48
토지면적(㎡)                        0.0
층                              9.0
권리구분                           NaN
취소일                            NaN
건축년도                        2004.0
건물용도                           아파트
신고구분                          중개거래
신고한 개업공인중개사 시군구명    서울 용산구, 서울 종로구
건물코드                  종로구-내수동-72-0
Name: 143, dtype: object

In [86]:
df_merge['건물코드'].str.split('-')

0      [강남구, 개포동, 1231, 21]
1       [강남구, 대치동, 1027, 0]
2        [강남구, 대치동, 316, 0]
3        [강남구, 도곡동, 467, 6]
4         [강남구, 도곡동, 91, 5]
               ...         
313      [중랑구, 신내동, 650, 0]
314      [중랑구, 신내동, 661, 0]
315      [중랑구, 신내동, 785, 0]
316      [중랑구, 신내동, 816, 0]
317      [중랑구, 중화동, 450, 0]
Name: 건물코드, Length: 318, dtype: object

In [71]:
pd.merge([df_prior_price, df_posterior_price], on='건물코드')

TypeError: merge() missing 1 required positional argument: 'right'

In [69]:
df_posterior_price

,건물코드,물건금액(만원)_posterior
0,강남구-개포동-1172-3,30500.0
1,강남구-개포동-1231-21,27500.0
2,강남구-개포동-13-3,19800.0
3,강남구-논현동-172-2,35000.0
4,강남구-논현동-22-0,87000.0
...,...,...
776,중랑구-중화동-309-106,25250.0
777,중랑구-중화동-322-34,30600.0
778,중랑구-중화동-330-100,36500.0
779,중랑구-중화동-450-0,54000.0
